In [1]:
!pip install wikipedia-api

In [3]:
!pip install firebase-admin

In [5]:
import pandas as pd
import requests

# Replace with your API keys
PEXELS_API_KEY = '4TVOWrMhOXc7KxDYD48wfrSMbDWty7kor11j5D7rB0Pae5TWT8CwQ9jK'
PIXABAY_API_KEY = '46982072-7531fb5ffc0361aecc8d872c7'
UNSPLASH_ACCESS_KEY = 'vTnfvmrRosNzqGHUFD4MJP1l_adh_b8bap1x9l0rM6A'

# Function to fetch image URL from Unsplash
def fetch_image_url_from_unsplash(location):
    search_url = f"https://api.unsplash.com/search/photos?query={location}&client_id={UNSPLASH_ACCESS_KEY}"
    response = requests.get(search_url)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch images from Unsplash for {location}.")
        return None
    
    data = response.json()
    if data['results']:
        image_url = data['results'][0]['urls']['regular']
        print(f"Image found for {location} on Unsplash: {image_url}")
        return image_url
    return None

# Function to fetch image URL from Pexels
def fetch_image_url_from_pexels(location):
    headers = {'Authorization': PEXELS_API_KEY}
    search_url = f"https://api.pexels.com/v1/search?query={location}&per_page=1"
    response = requests.get(search_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch images from Pexels for {location}.")
        return None
    
    data = response.json()
    if data['photos']:
        image_url = data['photos'][0]['src']['medium']
        print(f"Image found for {location} on Pexels: {image_url}")
        return image_url
    return None

# Function to fetch image URL from Pixabay
def fetch_image_url_from_pixabay(location):
    search_url = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={location}&image_type=photo"
    response = requests.get(search_url)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch images from Pixabay for {location}.")
        return None
    
    data = response.json()
    if data['hits']:
        image_url = data['hits'][0]['webformatURL']
        print(f"Image found for {location} on Pixabay: {image_url}")
        return image_url
    return None

# Function to fetch image URL from Wikimedia Commons
def fetch_image_url_from_wikimedia(location):
    user_agent = "stg/1.0 (akhnasj1612@gmail.com)"  # Replace with your app's name or email
    headers = {'User-Agent': user_agent}

    search_url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&titles={location}&prop=images"
    response = requests.get(search_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch images for {location} from Wikimedia Commons. Status code: {response.status_code}")
        return None
    
    data = response.json()
    page_id = list(data['query']['pages'].keys())[0]
    if page_id == "-1":
        print(f"Page for {location} not found on Wikimedia Commons.")
        return None
    
    images = data['query']['pages'][page_id].get('images', [])
    if not images:
        print(f"No images found for {location} on Wikimedia Commons.")
        return None
    
    image_title = images[0]['title']
    image_url = f"https://commons.wikimedia.org/wiki/Special:FilePath/{image_title.replace(' ', '_')}"
    print(f"Image found for {location} on Wikimedia Commons: {image_url}")
    return image_url

# Function to find image URL by trying each source in order of preference
def find_image_url(location):
    image_url = fetch_image_url_from_unsplash(location)
    if not image_url:
        image_url = fetch_image_url_from_pexels(location)
    if not image_url:
        image_url = fetch_image_url_from_pixabay(location)
    if not image_url:
        image_url = fetch_image_url_from_wikimedia(location)  # Wikimedia Commons as last resort
    return image_url or 'No Image Found'

# Load the locations CSV file
locations_df = pd.read_csv('C:\\Users\\AKHNA\\Desktop\\MCA\\Smart-Tour-Guide\\locations.csv', encoding='ISO-8859-1')

# Initialize an empty list to store the fetched image URLs
image_urls = []

# Loop through each location and fetch its image URL
for location in locations_df['Location']:  # Ensure 'Location' matches the column name in your CSV
    print(f"Fetching image for {location}...")
    image_url = find_image_url(location)
    image_urls.append(image_url)

# Add the image URLs as a new column to the dataframe
locations_df['Image_URL'] = image_urls

# Save the updated dataframe back to CSV
output_file = 'C:\\Users\\AKHNA\\Desktop\\MCA\\Smart-Tour-Guide\\loc.csv'
try:
    locations_df.to_csv(output_file, index=False)
    print(f"CSV file successfully updated. The file is saved as {output_file}")
except Exception as e:
    print(f"Error writing CSV file: {e}")


Fetching image for India Gate...
Error: Unable to fetch images from Unsplash for India Gate.
Image found for India Gate on Pexels: https://images.pexels.com/photos/789750/pexels-photo-789750.jpeg?auto=compress&cs=tinysrgb&h=350
Fetching image for Humayun's Tomb...
Error: Unable to fetch images from Unsplash for Humayun's Tomb.
Image found for Humayun's Tomb on Pexels: https://images.pexels.com/photos/24302006/pexels-photo-24302006.jpeg?auto=compress&cs=tinysrgb&h=350
Fetching image for Akshardham Temple...
Image found for Akshardham Temple on Unsplash: https://images.unsplash.com/photo-1671137777317-877064fb882d?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2NzM1Njd8MHwxfHNlYXJjaHwxfHxBa3NoYXJkaGFtJTIwVGVtcGxlfGVufDB8fHx8MTczMTE0Njg5M3ww&ixlib=rb-4.0.3&q=80&w=1080
Fetching image for Waste to Wonder Park...
Image found for Waste to Wonder Park on Unsplash: https://images.unsplash.com/photo-1686052903991-fedc0903ad4d?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2NzM1Njd8MHwxfHNlYXJj

In [ ]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase app
# Replace 'path/to/your/serviceAccountKey.json' with the path to your Firebase Admin SDK JSON file
cred = credentials.Certificate('C:\\Users\\AKHNA\\Downloads\\smart-tour-guide-3d30d-firebase-adminsdk-h17r2-bda949eabd.json')
firebase_admin.initialize_app(cred)

# Initialize Firestore client
db = firestore.client()

# Load the CSV with the updated image URLs
locations_df = pd.read_csv('C:\\Users\\AKHNA\\Desktop\\MCA\\Smart-Tour-Guide\\locations.csv', encoding='ISO-8859-1')

# Function to update Firestore 'locations' collection with new image URLs
def update_locations_with_new_urls():
    locations_ref = db.collection('locations')

    # Loop through each row in the DataFrame
    for _, row in locations_df.iterrows():
        location_name = row['Location']
        new_image_url = row['Image_URL']

        # Query to find the document by location name
        query = locations_ref.where('Location', '==', location_name).limit(1).stream()

        # Update the document with the new 'Image_URL' field if it has changed
        for doc in query:
            doc_ref = locations_ref.document(doc.id)
            doc_data = doc.to_dict()

            # Check if the image URL has changed
            if 'Image_URL' not in doc_data or doc_data['Image_URL'] != new_image_url:
                doc_ref.update({'Image_URL': new_image_url})
                print(f"Updated Image_URL for {location_name} with URL: {new_image_url}")
            else:
                print(f"No update needed for {location_name}, URL is already up-to-date.")

# Run the function to update Firestore documents
update_locations_with_new_urls()
print("Firestore 'locations' collection updated with new Image_URLs from locations.csv.")


C:\Users\AKHNA\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


Updated Image_URL for India Gate with URL: https://images.pexels.com/photos/789750/pexels-photo-789750.jpeg?auto=compress&cs=tinysrgb&h=350
Updated Image_URL for Humayun's Tomb with URL: https://images.pexels.com/photos/24302006/pexels-photo-24302006.jpeg?auto=compress&cs=tinysrgb&h=350
No update needed for Akshardham Temple, URL is already up-to-date.
No update needed for Waste to Wonder Park, URL is already up-to-date.
Updated Image_URL for Jantar Mantar with URL: https://images.pexels.com/photos/2302805/pexels-photo-2302805.jpeg?auto=compress&cs=tinysrgb&h=350
Updated Image_URL for Chandni Chowk with URL: https://images.pexels.com/photos/2344997/pexels-photo-2344997.jpeg?auto=compress&cs=tinysrgb&h=350
Updated Image_URL for Lotus Temple with URL: https://images.pexels.com/photos/1098460/pexels-photo-1098460.jpeg?auto=compress&cs=tinysrgb&h=350
Updated Image_URL for Red Fort with URL: https://images.pexels.com/photos/19528181/pexels-photo-19528181.jpeg?auto=compress&cs=tinysrgb&h=350